<div style="text-align:center;"><img src="http://www.mf-data-science.fr/images/projects/intro.jpg" style='width:100%; margin-left: auto; margin-right: auto; display: block;' /></div>

# <span style="color: #641E16">Contexte</span>
Nous allons ici développer un algorithme de Machine Learning destiné à assigner automatiquement plusieurs tags pertinents à une question posée sur le célébre site Stack overflow.     
Ce programme s'adresse principalement aux nouveaux utilisateurs, afin de leur suggérer quelques tags relatifs à la question qu'ils souhaitent poser.

### Les données sources
Les données ont été cleanées dans le Notebook Kaggle [Stackoverflow questions - data cleaning](https://www.kaggle.com/michaelfumery/stackoverflow-questions-data-cleaning). Dans ce nettoyage ont par exemple été appliquées les techniques de stop words, suppression de la ponctuation et des liens, tokenisation, lemmatisation ...

### Objectif de ce Notebook
Dans ce Notebook, nous allons traiter la partie **modélisation des données textuelles avec des modèles supervisés et non supervisés**.     

Tous les Notebooks du projet seront **versionnés dans Kaggle mais également dans un repo GitHub** disponible à l'adresse https://github.com/MikaData57/Analyses-donnees-textuelles-Stackoverflow

# <span style="color:#641E16">Sommaire</span>
1. [Preprocessing : Bag of Words / Tf-Idf](#section_1)
2. [Modèles non supervisés](#section_2)     
    2.1. [Modèle LDA](#section_2_1)     
    2.2. [Modèle NMF](#section_2_2)     
3. [Modèles supervisés](#section_3)     
    3.1. [Régression logistique avec multi-labels](#section_3_1)

In [ ]:
# Install package for PEP8 verification
!pip install pycodestyle
!pip install --index-url https://test.pypi.org/simple/ nbpep8

In [ ]:
# Import Python libraries
import os
import warnings
import time
import numpy as np
import pandas as pd
from ast import literal_eval
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models as gensimvis

# Library for PEP8 standard
from nbpep8.nbpep8 import pep8

In [ ]:
warnings.filterwarnings('ignore', category=DeprecationWarning)
plt.style.use('seaborn-whitegrid')
sns.set_style("whitegrid")
set_config(display='diagram')

In [ ]:
# Define path to data
path = '../input/stackoverflow-questions-filtered-2011-2021/'
data = pd.read_csv(path+"StackOverflow_questions_2009_2020_cleaned.csv",
                   sep=";", index_col=0,
                   converters={"Title": literal_eval,
                               "Body": literal_eval,
                               "Tags": literal_eval})
data.head(3)

In [ ]:
data.shape

Nous allons également créer une variable `Full_doc` qui accueillera le document complet de chaque item (Title et Body) :

In [ ]:
data["Full_doc"] = data["Title"] + data["Body"]
data["Full_doc"].head(3)

# <span style="color:#641E16" id="section_1">Preprocessing : Bag of Words / Tf-Idf</span>

Pour alimenter les modèles de machine learning, nous avons besoin de traiter des données numériques. Le modèle **Bag of Words** apprend un vocabulaire à partir de tous les documents, puis modélise chaque document en comptant le nombre de fois où chaque mot apparaît, convertissant donc les données textuelles en données numériques.

Nos données ayant déjà été cleanées et tokenisées dans le Notebook [stackoverflow-questions-data-cleaning](https://www.kaggle.com/michaelfumery/stackoverflow-questions-data-cleaning), nous allons initialiser l'algorithme du `CountVectorizer` sur les variables `Title` et `Body` *(X1 et X2)* sans preprocessing. Enfin, nous allons utiliser le module `TfidfVectorizer` de la librairie Scikit-Learn pour combiner le `CountVectorizer` et `TfidfTransformer`. Cela aura pour effet de pondérer la fréquence d'apparition des mots par un indicateur de similarité *(si ce mot est commun ou rare dans tous les documents)*. Dans cette partie, nous allons **éliminer les mots qui apparaissent dans plus de 60% des documents** (`max_df = 0.6`).

la métrique tf-idf ***(Term-Frequency - Inverse Document Frequency)*** utilise comme indicateur de similarité l'inverse document frequency qui est l'inverse de la proportion de document qui contient le terme, à l'échelle logarithmique.

Pour préparer nos targets *(pour les modèles supervisés)*, nous allons utiliser `MultiLabelBinarizer` de Scikit-Learn puisque nos `Tags` sont multiples.

In [ ]:
# Create train and test split (30%)
X = data[["Title", "Body", "Full_doc"]]
y = data["Tags"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

# Initialize the "CountVectorizer" object for Title, Body and Full_doc
title_vectorizer = TfidfVectorizer(analyzer="word",
                                   max_df=.6,
                                   min_df=0.005,
                                   tokenizer=None,
                                   preprocessor=' '.join,
                                   stop_words=None,
                                   lowercase=False)

body_vectorizer = TfidfVectorizer(analyzer="word",
                                  max_df=.6,
                                  min_df=0.005,
                                  tokenizer=None,
                                  preprocessor=' '.join,
                                  stop_words=None,
                                  lowercase=False)

full_vectorizer = TfidfVectorizer(analyzer="word",
                                  max_df=.6,
                                  min_df=0.005,
                                  tokenizer=None,
                                  preprocessor=' '.join,
                                  stop_words=None,
                                  lowercase=False)

Xt_tfidf = title_vectorizer.fit_transform(X_train["Title"])
Xb_tfidf = body_vectorizer.fit_transform(X_train["Body"])
X_tfidf = full_vectorizer.fit_transform(X_train["Full_doc"])

print("Shape of X for Title: {}".format(Xt_tfidf.shape))
print("Shape of X for Body: {}".format(Xb_tfidf.shape))
print("Shape of X for Full_doc: {}".format(X_tfidf.shape))

# Multilabel binarizer for targets
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(y_train)
y_binarized = multilabel_binarizer.transform(y_train)
y_test_binarized = multilabel_binarizer.transform(y_test)
print("Shape of y: {}".format(y_binarized.shape))

Comme les matrices sont relativment importantes, nous allons **vérifier le nombre de cellules qui ne sont pas à 0** :

In [ ]:
title_dense = Xt_tfidf.todense()
body_dense = Xb_tfidf.todense()
full_dense = X_tfidf.todense()
print("Title sparsicity: {:.3f} %"\
      .format(((title_dense > 0).sum()/title_dense.size)*100))
print("Body sparsicity: {:.3f} %"\
      .format(((body_dense > 0).sum()/body_dense.size)*100))
print("Full_doc sparsicity: {:.3f} %"\
      .format(((full_dense > 0).sum()/full_dense.size)*100))

Nous constatons que cette mesure est meilleure pour la varaible englobant Title et Body *(Full_doc)*.

# <span style="color:#641E16" id="section_2">Modèles non supervisés</span>

## <span id="section_2_1">Modèle LDA</span>
LDA, ou **Latent Derelicht Analysis** est un modèle probabiliste qui, pour obtenir des affectations de cluster, utilise deux valeurs de probabilité : $P(word | topics)$ et $P(topics | documents)$. Ces valeurs sont calculées sur la base d'une attribution aléatoire initiale, puis le calcul est répété pour chaque mot dans chaque document, pour décider de leur attribution de sujet. Dans cette méthode itérative, ces probabilités sont calculées plusieurs fois, jusqu'à la convergence de l'algorithme.

Nous allons entrainer 1 seul modèle basé sur la variable `Full_doc` en utilisant la librairie spécialisée **Gensim**. Pour cette partie, nous n'utiliserons pas le preprocessing TFIDF mais des fonctions propres aux méthodes Gensim.

Dans une première étape, le Bag of words est créé ainsi que la matrice de fréquence des termes dans les documents :

In [ ]:
# Create dictionnary (bag of words)
id2word = corpora.Dictionary(X_train.Full_doc)
id2word.filter_extremes(no_below=4, no_above=0.6, keep_n=None)
# Create Corpus 
texts = X_train.Full_doc  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

Gensim crée un identifiant unique pour chaque mot du document puis mappe word_id et word_frequency. Exemple : (6,3) ci-dessus indique que word_id 6 apparaît 3 fois dans le document et ainsi de suite.      
Les mots les plus fréquents ont ici aussi été filtrés grâce à la fonction `filter_extremes` réglée à 60% comme pour le Tfidf.

Pour voir quel mot correspond à un identifiant donné, il faut transmettre l'identifiant comme clé du dictionnaire. Exemple : id2word[4] :

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

Nous allons à présent entrainer le modèle LDA sur Full_doc puis afficher les métriques : 
- **Perplexity** : $(\exp(-1 \times \text{log-likelihood})$ *(Log likelihood : Densité de vraisemblance)*
- **Coherence Score** : Les mesures de cohérence de topics évaluent un seul topic en mesurant le degré de similitude sémantique entre les mots à score élevé dans ce dernier.       Pour en savoir plus sur ce Pipeline : [What is topic coherence ?](https://rare-technologies.com/what-is-topic-coherence/)

In [ ]:
# Build LDA model
full_lda_model = gensim.models.ldamulticore\
                    .LdaMulticore(corpus=corpus,
                                  id2word=id2word,
                                  num_topics=20,
                                  random_state=8,
                                  per_word_topics=True,
                                  workers=4)
# Print Perplexity score
print('\nPerplexity: ', full_lda_model.log_perplexity(corpus))

#Print Coherence Score
coherence_model_lda = CoherenceModel(model=full_lda_model, 
                                     texts=texts, 
                                     dictionary=id2word, 
                                     coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

### <span style="color:#641E16;">Visualisation des résultats de LDA Gensim sur Full_doc avec 20 topics</span>

In [ ]:
pyLDAvis.enable_notebook()
gensimvis.prepare(full_lda_model, corpus, id2word)

D'après les résulatas de cette modélisation LDA, il semble très **difficile de "nommer" les topics créés car les mots qui les composent sont très variés et sans fil conducteur clairement établi**. On voit cependant par exemple que le topic représenté par "server" englobe également "database", "sql", "connection" ou encore "query" ce qui est cohérent.

### Amélioration du modèle LDA

Cependant, dans l'algoritmes LDA, nous avons fixé arbitrairement à 20 le paramètre `num_topics` qui représente le nombre de topics à créer. Afin de sélectionner le meilleur nombre de topics pour nos données, nous allons **itérer sur une fourchette de nombre de topics et tester le score de cohérence pour chaque modèle** :

In [ ]:
# Iter LDA for best number of topics
coherence_test = []
for k in np.arange(1,90,10):
    print("Fitting LDA for K = {}".format(k))
    start_time = time.time()
    lda_model = gensim.models.ldamulticore\
                    .LdaMulticore(corpus=corpus,
                                  id2word=id2word,
                                  num_topics=k,
                                  random_state=8,
                                  per_word_topics=True,
                                  workers=4)
    coherence_model_lda = CoherenceModel(model=lda_model,
                                         texts=texts,
                                         dictionary=id2word,
                                         coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    end_time = time.time()
    coherence_test.append((k, coherence_lda, (end_time - start_time)))

Affichons les scores des divers modèle :

In [ ]:
# Create dataframe of results
coherence_test = pd.DataFrame(coherence_test,
                              columns=["k","coherence","time"])

# Select best number of topics
best_nb_topics = coherence_test\
                    .loc[coherence_test.coherence.argmax(),"k"]

# Plot results
fig, ax1 = plt.subplots(figsize=(12,8))
x = coherence_test["k"]
y1 = coherence_test["coherence"]
y2 = coherence_test["time"]

ax1.plot(x, y1, label="Coherence score")
ax1.axvline(x=best_nb_topics, color='r', alpha=.7,
            linestyle='dashdot', label='Best param')
ax1.set_xlabel("Number of components")
ax1.set_ylabel("Coherence score")

ax2 = ax1.twinx()
ax2.plot(x, y2, label="Fit time",
         color='g', alpha=.5,
         linestyle='--')
ax2.set_ylabel("Fitting time (s)")

plt.title("Choosing Optimal LDA Model\n",
          color="#641E16", fontsize=18)
legend = fig.legend(loc=1, bbox_to_anchor=(.92, .9))

fig.tight_layout()
plt.show()

Testons à présent le modèle avec le meilleur nombre théorique de topics pour l'afficher avec LDAvis :

In [ ]:
# Best LDA visualization
# Construire le modèle LDA
best_lda_model = gensim.models.ldamulticore\
                    .LdaMulticore(corpus=corpus,
                                  id2word=id2word,
                                  num_topics=best_nb_topics,
                                  random_state=8,
                                  per_word_topics=True,
                                  workers=4)
gensimvis.prepare(best_lda_model, corpus, id2word)

Pour attribuer des Tags à chaque question sur ces modèles non-supervisés, nous allons **créer une matrice Topic/Tags** en réalisant une multiplication matricielle des matrices Document / Topic et Document / Tags.

In [ ]:
# Calculate Document/topic matrix with Gensim
doc_topic = pd.DataFrame(best_lda_model.get_document_topics(corpus, minimum_probability=0))
for topic in doc_topic.columns:
    doc_topic[topic] = doc_topic[topic].apply(lambda x : x[1])

print('document/tag : ', y_binarized.shape)
print('document/topic : ', doc_topic.shape)

In [ ]:
# Print documents / topic matrix
doc_topic.head(3)

A présent, créons la matrice Topic / Tags grâce aux probabilités obtenues :

In [ ]:
# Matricial multiplication with Document / Topics transpose
topic_tag = np.matmul(doc_topic.T, y_binarized)
topic_tag.shape

In [ ]:
topic_tag

Nous obtenons donc une matrice dont les lignes représentent les Topics créés et les colonnes les Tags associés et leurs distribution. Nous allons donc **créer nos prédictions en prenant les** $\large n$ **premiers tags associés aux topics** de chaque document :

In [ ]:
y_results = pd.DataFrame(y_train)
y_results["best_topic"] = doc_topic.idxmax(axis=1).values
y_results["nb_tags"] = y_results["Tags"].apply(lambda x : len(x))

df_y_bin = pd.DataFrame(y_binarized)
df_dict = dict(
    list(
        df_y_bin.groupby(df_y_bin.index)
    )
)

tags_num = []
for k, v in df_dict.items():
    check = v.columns[(v == 1).any()]
    tags_num.append(check.to_list())

y_results["y_true"] = tags_num
y_results.head(3)

In [ ]:
# Select predicted tags in Topics / Tags matrix
list_tag = []
for row in y_results.itertuples():
    nb_tags = row.nb_tags
    best_topic = row.best_topic
    row_tags = list(topic_tag.iloc[best_topic].sort_values(ascending=False)[0:nb_tags].index)
    list_tag.append(row_tags)
    
y_results["y_pred"] = list_tag
y_results.head(3)

Nous allons tester plusieurs métriques sur ce modèle LDA :
- Accuracy score :
- F1 score :
- Jaccard similarity score : 
- Recall :
- Precision :

In [ ]:
def metrics_score(model, df, y_true, y_pred):
    if(df is not None):
        temp_df = df
    else:
        temp_df = pd.DataFrame(index=["Accuracy", "F1", "Jaccard", "Recall", "Precision"],
                               columns=[model])
        
    scores = []
    scores.append(metrics.accuracy_score(y_true, y_pred))
    scores.append(metrics.f1_score(y_pred, y_true, average='weighted'))
    scores.append(metrics.jaccard_score(y_true, y_pred, average='weighted'))
    scores.append(metrics.recall_score(y_true, y_pred, average='weighted'))
    scores.append(metrics.precision_score(y_true, y_pred, average='weighted'))
    temp_df[model] = scores
    
    return temp_df

metrics_score("LDA", df=None, 
              y_true=y_results.y_true, 
              y_pred=y_results.y_pred)

On remarque ici que la modélisation non supervisée avec LDA n'est pas adaptée. En effet, le meilleur nombre de topics se situerait à 31, mais **l'algorithme ne parvient pas a établir de groupes bien distincts**. Un certain nombre de topics sont très regroupés et donc représentés par les mêmes termes.

Nous allons donc tester une seconde modélisation non supervisée.

## <span id="section_2_2">Modèle NMF</span>
<details>
  <summary style="color:blue;">Explication du modèle</summary>
  
  ## NMF
  La factorisation matricielle non négative *(**N**on-negative **M**atrix **F**actorization)* est un modèle linéaire-algéabrique, qui factorise des vecteurs de grande dimension dans une représentation de faible dimension. Similaire à l'analyse en composantes principales *(PCA)*, NMF profite du fait que **les vecteurs sont non négatifs**. En les factorisant dans la forme de dimension inférieure, NMF force les coefficients à être également non négatifs.<br/><br/>
    Prenons une matrice d'origine $A$, nous pouvons obtenir deux matrices $W$ et $H$, telles que $A = WH$. NMF a une propriété de clustering, telle que $W$ et $H$ représentent les informations suivantes sur $A$ :
    <ul><li>$A$ (Matrice Document-word) : Matrice qui contient "quels mots apparaissent dans quels documents".</li>
    <li>$W$ (Vecteurs de base) : Topics découverts à partir des documents.</li>
    <li>$H$ (Matrice de coefficients) : les poids pour les topics dans chaque document.</li></ul><br/>
    Nous calculons $W$ et $H$ en optimisant sur une **fonction objectif**, en mettant à jour à la fois $W$ et $H$ de manière itérative jusqu'à convergence.</br></br>
    $$\large \frac{1}{2} ||A - WH||^2_F = \sum_{i=1}^{n} \sum_{j=1}^{m} (A_{ij} - (WH)_{ij})^2$$</br>
    Dans cette fonction objectif, nous mesurons l'erreur de reconstruction entre A et le produit de ses facteurs W et H, en fonction de la distance euclidienne. Les valeurs mises à jour sont calculées dans des opérations parallèles, et en utilisant les nouveaux W et H, nous recalculons l'erreur de reconstruction, en répétant ce processus jusqu'à la convergence.
</details>

Le modèle **NMF ne peut malheureusement pas être scoré**. Nous allons donc nous baser sur les résultats de la LDA pour déterminer un nombre correct de composants. Ici, nous prendrons **12 topics** pour avoir un bon compromis "temps d'entrainement" / précision et utiliserons les matrices Tfidf créées lors du preprocessing.

In [ ]:
# Define number of topics to test
n_topics = best_nb_topics

print("-"*50)
print("Start NMF fitting on Title ...")
print("-" * 50)
start_time = time.time()
# Initializing the NMF
title_nmf = NMF(n_components=n_topics,
                init='nndsvd',
                random_state=8)

# Fit NMF on Title vectorized
title_nmf.fit(Xt_tfidf)

exec_time = time.time() - start_time
print("End of training :")
print("Execution time : {:.2f}s".format(exec_time))
print("-" * 50)

L'entrainement est plus rapide que pour LDA. Visualisons à présent les résulats en plotant **les 20 meilleurs mots de chaque topic**.

In [ ]:
def plot_top_words(model, feature_names, n_top_words, nb_topic_plot, title):
    """Function for displaying the plots of the 
    best x words representative of the categories of NMF.

    Parameters
    ----------------------------------------
    model : NMF model
        Fitted model of NMF to plot
    feature_names : array
        Categories result of the vectorizer (TFIDF ...)
    n_top_words : int
        Number of words for each topic.
    title : string
        Title of the plot.
    ----------------------------------------
    """
    rows = int(nb_topic_plot/6)
    fig, axes = plt.subplots(rows, 6, 
                             figsize=(30, rows*10), 
                             sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        if(topic_idx < nb_topic_plot):
            top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
            top_features = [feature_names[i] for i in top_features_ind]
            weights = topic[top_features_ind]

            ax = axes[topic_idx]
            bartopic = ax.barh(top_features, weights, height=0.7)
            bartopic[0].set_color('#f48023')
            ax.set_title(f'Topic {topic_idx +1}',
                         fontdict={'fontsize': 30})
            ax.invert_yaxis()
            ax.tick_params(axis='both', which='major', labelsize=20)
            for i in 'top right left'.split():
                ax.spines[i].set_visible(False)
            fig.suptitle(title, fontsize=36, color="#641E16")

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

In [ ]:
# Plot the 12 first topics
tf_feature_names = title_vectorizer.get_feature_names()
plot_top_words(title_nmf, tf_feature_names, 20, 12,
               'Topics in NMF model for Title feature')

**La modélisation avec NMF nous apporte des catégories plus lisibles que celles de l'algorithme LDA**. 1 mot est toujours beaucoup plus représentaif de cette catégorie mais les regroupements sont globalement cohérents. Un topic par exemple illustre bien les sujets liés à Python. un second traite les sujets liés à Android ...

Nous allons **tester cette modélisation sur la variable Body** cette fois :

In [ ]:
print("-"*50)
print("Start NMF fitting on Body ...")
print("-" * 50)
start_time = time.time()
# Initializing the NMF
body_nmf = NMF(n_components=n_topics,
               init='nndsvd',
               random_state=8)

# Fit NMF on Body vectorized
body_nmf.fit(Xb_tfidf)

exec_time = time.time() - start_time
print("End of training :")
print("Execution time : {:.2f}s".format(exec_time))
print("-" * 50)

bf_feature_names = body_vectorizer.get_feature_names()
plot_top_words(body_nmf, bf_feature_names, 20, 12,
               'Topics in NMF model for Body feature')

Basé sur le corpus Body, le modèle fait sortir des thèmes différents, plus génériques. **NMF est également beaucoup plus rapide que LDA**, ce qui est un atout pour en terme de puissance de calcul nécessaire.

In [ ]:
print("-"*50)
print("Start NMF fitting on Full_doc ...")
print("-" * 50)
start_time = time.time()
# Initializing the NMF
full_nmf = NMF(n_components=n_topics,
               init='nndsvd',
               random_state=8)

# Fit NMF on Body vectorized
full_nmf.fit(X_tfidf)

exec_time = time.time() - start_time
print("End of training :")
print("Execution time : {:.2f}s".format(exec_time))
print("-" * 50)

ff_feature_names = full_vectorizer.get_feature_names()
plot_top_words(full_nmf, ff_feature_names, 20, 12,
               'Topics in NMF model for Full_doc')

La modélisation NMF sur la variable `Full_doc` semble offrir de meilleurs résultats dans la contribution des termes aux topics. Nous conserverons donc cette variable pour la suite des analyses. 

# <span style="color:#641E16" id="section_3">Modèles supervisés</span>

## <span id="section_3_1">Régression logistique avec multi-labels</span>
Nous avons déjà réalisé quelques Notebook traitant de la régression logistique : c'est une technique prédictive. Elle vise à construire un modèle permettant de prédire / expliquer les valeurs prises par une variable cible qualitative à partir d’un ensemble de variables explicatives quantitatives ou qualitatives encodées.

Pour cette partie sur les modélisations supervisées, nous allons utiliser la variable `Full_doc` qui regroupe le Title et le Body puis créer un Pipeline qui ne pourra pas inclure la transformation de notre variable cible *(MultiLabelBinarizer ne fonctionne pas dans les Pipeline SKlearn)*.

In [ ]:
# Fixe X to Full_doc
X_train = X_train['Full_doc']
X_test = X_test['Full_doc']

# create pipeline with preprocessing
mlb = MultiLabelBinarizer()
y_train_mlb = mlb.fit_transform(y_train)
y_test_mlb = mlb.fit_transform(y_test)
multi_logit = Pipeline([("vectorizer", TfidfVectorizer(analyzer="word",
                                                       max_df=.6,
                                                       min_df=0.005,
                                                       tokenizer=None,
                                                       preprocessor=' '.join,
                                                       stop_words=None,
                                                       lowercase=False)),
                        ("logit", OneVsRestClassifier(LogisticRegression(C=1.0, 
                                                                         penalty='l1', 
                                                                         dual=False, 
                                                                         solver='liblinear')))])
multi_logit.fit(X_train, y_train_mlb)

Nous pouvons maintenant **réaliser les prédictions avec le modèle de régression logistique sur le jeu de test** pour pouvoir les comparer avec le jeu y_test.

In [ ]:
# Predict
y_test_predicted_labels_tfidf = multi_logit.predict(X_test)

# Inverse transform
y_test_pred_inversed = mlb.inverse_transform(y_test_predicted_labels_tfidf)
y_test_inversed = mlb.inverse_transform(y_test_mlb)

In [ ]:
df_metrics_compare = metrics_score("Logit", df=None, 
              y_true = y_test_mlb, 
              y_pred = y_test_predicted_labels_tfidf)
df_metrics_compare

## <span id="section_3_2">Modélisation avec RandomForest</span>

In [ ]:
# create pipeline with preprocessing
multi_rfc = Pipeline([("vectorizer", TfidfVectorizer(analyzer="word",
                                                       max_df=.6,
                                                       min_df=0.005,
                                                       tokenizer=None,
                                                       preprocessor=' '.join,
                                                       stop_words=None,
                                                       lowercase=False)),
                        ("randomForest", OneVsRestClassifier(RandomForestClassifier(n_jobs=-1)))])
multi_rfc.fit(X_train, y_train_mlb)

In [ ]:
# Predict
y_test_predicted_labels_tfidf_rfc = multi_rfc.predict(X_test)

df_metrics_compare = metrics_score("RandomForest", df=df_metrics_compare, 
              y_true = y_test_mlb, 
              y_pred = y_test_predicted_labels_tfidf_rfc)
df_metrics_compare